# DeepSeek V3 Function Call 测试与实现

这个 notebook 演示了如何使用 DeepSeek V3 模型实现传统的 function calling 功能。

## Function Call 说明

Function Call 是 OpenAI API 的传统功能，虽然现在推荐使用 Tool Use，但 Function Call 仍然被广泛支持和使用。

In [19]:
import asyncio
import json
import os
from dotenv import load_dotenv
from fallback_openai_client import AsyncFallbackOpenAIClient

# 加载环境变量
load_dotenv()

# 配置 DeepSeek API
DEEPSEEK_API_KEY = os.getenv("API_KEY")
DEEPSEEK_BASE_URL = os.getenv("BASE_URL") 
DEEPSEEK_MODEL = "deepseek-v3-250324"

print(f"API Key: {DEEPSEEK_API_KEY[:20]}..." if DEEPSEEK_API_KEY else "API Key not found")
print(f"Base URL: {DEEPSEEK_BASE_URL}")
print(f"Model: {DEEPSEEK_MODEL}")

API Key: 7b448f0e-e6cb-421e-9...
Base URL: https://ark.cn-beijing.volces.com/api/v3
Model: deepseek-v3-250324


## 1. 测试基本 API 连接

首先测试 DeepSeek API 是否能正常调用。

In [20]:
async def test_basic_api():
    """测试基本的 API 连接"""
    client = AsyncFallbackOpenAIClient(
        primary_api_key=DEEPSEEK_API_KEY,
        primary_base_url=DEEPSEEK_BASE_URL,
        primary_model_name=DEEPSEEK_MODEL
    )
    
    try:
        response = await client.chat_completions_create(
            messages=[
                {"role": "user", "content": "你是谁？"}
            ],
            max_tokens=100,
            temperature=0.7
        )
        
        print("✅ API 连接成功!")
        print(f"响应: {response.choices[0].message.content}")
        return True
        
    except Exception as e:
        print(f"❌ API 连接失败: {e}")
        return False
    
    finally:
        await client.close()

# 运行测试
await test_basic_api()

⚠️ 警告: 未完全配置备用 API 客户端。如果主 API 失败，将无法进行回退。
✅ API 连接成功!
响应: 我是DeepSeek Chat，由深度求索公司创造的智能AI助手！🤖✨ 我可以回答你的问题、帮你查找信息、提供学习或工作上的建议，甚至陪你聊天解闷儿~ 有什么我可以帮你的吗？😊
✅ API 连接成功!
响应: 我是DeepSeek Chat，由深度求索公司创造的智能AI助手！🤖✨ 我可以回答你的问题、帮你查找信息、提供学习或工作上的建议，甚至陪你聊天解闷儿~ 有什么我可以帮你的吗？😊


True

## 2. 定义 Function Call 函数

定义一些示例函数供模型调用。

In [21]:
import math
from datetime import datetime

# 定义可供调用的函数
def get_current_time():
    """获取当前时间"""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def calculate_circle_area(radius: float):
    """计算圆的面积"""
    if radius <= 0:
        return {"error": "半径必须大于0"}
    return {"area": math.pi * radius ** 2}

def get_weather_info(city: str):
    """获取天气信息（模拟）"""
    weather_data = {
        "北京": "晴天，温度 15°C",
        "上海": "多云，温度 18°C", 
        "广州": "雨天，温度 22°C",
        "深圳": "晴天，温度 25°C"
    }
    return weather_data.get(city, f"抱歉，暂无 {city} 的天气信息")

def search_information(query: str):
    """搜索信息（模拟）"""
    search_results = {
        "Python": "Python 是一种高级编程语言，由 Guido van Rossum 创建。",
        "AI": "人工智能是计算机科学的一个分支，致力于创建能够模拟人类智能的系统。",
        "机器学习": "机器学习是人工智能的一个子集，通过数据训练算法来做出预测或决策。"
    }
    
    for key, value in search_results.items():
        if key.lower() in query.lower():
            return value
    
    return f"抱歉，没有找到关于 '{query}' 的相关信息。"

# 函数映射表
AVAILABLE_FUNCTIONS = {
    "get_current_time": get_current_time,
    "calculate_circle_area": calculate_circle_area,
    "get_weather_info": get_weather_info,
    "search_information": search_information
}

print("✅ 函数定义完成")

✅ 函数定义完成


## 3. Function Call 函数定义

为传统的 Function Call API 格式定义函数规范。

In [22]:
# 定义函数规范（传统的 Function Call 格式）
FUNCTIONS = [
    {
        "name": "get_current_time",
        "description": "获取当前的日期和时间",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "calculate_circle_area",
        "description": "根据半径计算圆的面积",
        "parameters": {
            "type": "object",
            "properties": {
                "radius": {
                    "type": "number",
                    "description": "圆的半径，必须大于0"
                }
            },
            "required": ["radius"]
        }
    },
    {
        "name": "get_weather_info", 
        "description": "获取指定城市的天气信息",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "城市名称，如：北京、上海、广州、深圳"
                }
            },
            "required": ["city"]
        }
    },
    {
        "name": "search_information",
        "description": "搜索相关信息",
        "parameters": {
            "type": "object", 
            "properties": {
                "query": {
                    "type": "string",
                    "description": "要搜索的关键词或问题"
                }
            },
            "required": ["query"]
        }
    }
]

print(f"✅ 定义了 {len(FUNCTIONS)} 个函数")
for func in FUNCTIONS:
    print(f"  - {func['name']}: {func['description']}")

✅ 定义了 4 个函数
  - get_current_time: 获取当前的日期和时间
  - calculate_circle_area: 根据半径计算圆的面积
  - get_weather_info: 获取指定城市的天气信息
  - search_information: 搜索相关信息


## 4. Function Call 核心实现

实现 function call 的核心逻辑。

In [23]:
class DeepSeekFunctionCaller:
    """DeepSeek Function Call 实现类"""
    
    def __init__(self, api_key: str, base_url: str, model: str):
        self.client = AsyncFallbackOpenAIClient(
            primary_api_key=api_key,
            primary_base_url=base_url,
            primary_model_name=model
        )
        self.functions = FUNCTIONS
        self.available_functions = AVAILABLE_FUNCTIONS
    
    async def execute_function_call(self, function_name: str, arguments: dict):
        """执行函数调用"""
        try:
            if function_name not in self.available_functions:
                return {"error": f"函数 {function_name} 不存在"}
            
            func = self.available_functions[function_name]
            
            # 执行函数
            if arguments:
                result = func(**arguments)
            else:
                result = func()
            
            return {"success": True, "result": result}
            
        except Exception as e:
            return {"success": False, "error": str(e)}
    
    async def chat_with_functions(self, messages: list, max_function_calls: int = 5):
        """支持 function call 的对话"""
        conversation_messages = messages.copy()
        function_call_count = 0
        
        while function_call_count < max_function_calls:
            try:
                # 调用 API
                response = await self.client.chat_completions_create(
                    messages=conversation_messages,
                    functions=self.functions,
                    function_call="auto",
                    temperature=0.7,
                    max_tokens=1000
                )
                
                message = response.choices[0].message
                
                # 检查是否有 function call
                if message.function_call:
                    print(f"🔧 检测到函数调用: {message.function_call.name}")
                    
                    function_name = message.function_call.name
                    try:
                        function_args = json.loads(message.function_call.arguments)
                    except json.JSONDecodeError:
                        function_args = {}
                    
                    print(f"📞 调用函数: {function_name}")
                    print(f"📝 参数: {function_args}")
                    
                    # 执行函数
                    function_result = await self.execute_function_call(function_name, function_args)
                    
                    print(f"✅ 函数执行结果: {function_result}")
                    
                    # 添加助手消息和函数结果到对话
                    conversation_messages.append({
                        "role": "assistant",
                        "content": None,
                        "function_call": {
                            "name": function_name,
                            "arguments": message.function_call.arguments
                        }
                    })
                    
                    conversation_messages.append({
                        "role": "function",
                        "name": function_name,
                        "content": json.dumps(function_result, ensure_ascii=False)
                    })
                    
                    function_call_count += 1
                    continue  # 继续下一轮对话
                
                else:
                    # 没有 function call，返回最终响应
                    return {
                        "response": message.content,
                        "conversation": conversation_messages,
                        "function_calls_used": function_call_count
                    }
                    
            except Exception as e:
                return {
                    "error": f"API 调用失败: {e}",
                    "conversation": conversation_messages,
                    "function_calls_used": function_call_count
                }
        
        # 达到最大函数调用次数
        return {
            "response": "已达到最大函数调用次数限制",
            "conversation": conversation_messages,
            "function_calls_used": function_call_count
        }
    
    async def close(self):
        """关闭客户端"""
        await self.client.close()

print("✅ DeepSeekFunctionCaller 类定义完成")

✅ DeepSeekFunctionCaller 类定义完成


## 5. 测试 Function Call 功能

测试各种 function call 场景。

In [24]:
async def test_function_calls():
    """测试 function call 功能"""
    
    caller = DeepSeekFunctionCaller(
        api_key=DEEPSEEK_API_KEY,
        base_url=DEEPSEEK_BASE_URL, 
        model=DEEPSEEK_MODEL
    )
    
    test_cases = [
        "现在几点了？",
        "帮我计算半径为5的圆的面积",
        "北京今天天气怎么样？",
        "搜索一下关于Python的信息",
        "现在时间是多少，然后告诉我广州的天气情况"
    ]
    
    try:
        for i, question in enumerate(test_cases, 1):
            print(f"\n{'='*60}")
            print(f"测试用例 {i}: {question}")
            print('='*60)
            
            messages = [{"role": "user", "content": question}]
            
            result = await caller.chat_with_functions(messages)
            
            if "error" in result:
                print(f"❌ 错误: {result['error']}")
            else:
                print(f"🤖 AI 响应: {result['response']}")
                print(f"📊 使用了 {result['function_calls_used']} 次函数调用")
            
            # 添加分隔符
            print("-" * 60)
    
    finally:
        await caller.close()

# 运行测试
await test_function_calls()

⚠️ 警告: 未完全配置备用 API 客户端。如果主 API 失败，将无法进行回退。

测试用例 1: 现在几点了？
🤖 AI 响应: 我无法获取实时时间，但你可以查看手机、电脑或手表的时钟功能来获取当前准确时间。如果你需要其他帮助，比如时区转换或时间管理建议，我很乐意帮忙！ 😊
📊 使用了 0 次函数调用
------------------------------------------------------------

测试用例 2: 帮我计算半径为5的圆的面积
🤖 AI 响应: 我无法获取实时时间，但你可以查看手机、电脑或手表的时钟功能来获取当前准确时间。如果你需要其他帮助，比如时区转换或时间管理建议，我很乐意帮忙！ 😊
📊 使用了 0 次函数调用
------------------------------------------------------------

测试用例 2: 帮我计算半径为5的圆的面积
🤖 AI 响应: 要计算半径为 \(5\) 的圆的面积，可以使用圆的面积公式：

\[
\text{面积} = \pi r^2
\]

其中：
- \(r\) 是圆的半径，
- \(\pi\) 是圆周率，约等于 \(3.1416\)。

**步骤如下：**

1. **将半径代入公式：**
   
   \[
   \text{面积} = \pi \times 5^2
   \]

2. **计算半径的平方：**
   
   \[
   5^2 = 25
   \]
   
   所以，
   
   \[
   \text{面积} = \pi \times 25
   \]

3. **乘以圆周率 \(\pi\)：**
   
   \[
   \text{面积} \approx 3.1416 \times 25
   \]
   
   \[
   \text{面积} \approx 78.54
   \]

**最终答案：**

\[
\boxed{25\pi} \quad \text{或} \quad \boxed{78.54} \quad \text{（取近似值）}
\]

因此，半径为 \(5\) 的圆的面积是 \(25\pi\) 平方单位，约等于 \(78.54\) 平方单位。
📊 使用了 0 次函数调用
-----------

## 6. 交互式对话测试

创建一个简单的交互式对话示例。

In [ ]:
async def interactive_chat_demo():
    """交互式对话演示"""
    
    caller = DeepSeekFunctionCaller(
        api_key=DEEPSEEK_API_KEY,
        base_url=DEEPSEEK_BASE_URL,
        model=DEEPSEEK_MODEL
    )
    
    # 预设一些演示问题
    demo_questions = [
        "你好！你可以帮我做什么？",
        "现在几点了？顺便告诉我深圳的天气",
        "计算一下半径为10的圆的面积",
        "搜索关于机器学习的信息"
    ]
    
    conversation_history = []
    
    try:
        print("🚀 开始交互式对话演示")
        print("📱 可用功能：获取时间、计算圆面积、查询天气、搜索信息")
        print("=" * 60)
        
        for question in demo_questions:
            print(f"\n👤 用户: {question}")
            
            # 添加用户消息到历史
            conversation_history.append({"role": "user", "content": question})
            
            # 获取 AI 响应
            result = await caller.chat_with_functions(conversation_history.copy())
            
            if "error" in result:
                print(f"❌ 错误: {result['error']}")
            else:
                print(f"🤖 助手: {result['response']}")
                
                # 更新对话历史（只保留用户和助手的主要对话）
                conversation_history.append({
                    "role": "assistant", 
                    "content": result['response']
                })
                
                if result['function_calls_used'] > 0:
                    print(f"🔧 (使用了 {result['function_calls_used']} 次函数调用)")
    
    finally:
        await caller.close()
        print(f"\n✅ 演示结束")

# 运行交互式演示
await interactive_chat_demo()

⚠️ 警告: 未完全配置备用 API 客户端。如果主 API 失败，将无法进行回退。
🚀 开始交互式对话演示
📱 可用功能：获取时间、计算圆面积、查询天气、搜索信息

👤 用户: 你好！你可以帮我做什么？
🤖 助手: 你好！我可以帮助你完成很多事情，以下是一些我可以提供的服务：

1. **信息查询**：
   - 获取当前的日期和时间。
   - 查询天气信息（告诉我城市名称即可）。
   - 搜索互联网上的相关信息（比如新闻、百科等）。

2. **计算工具**：
   - 计算圆的面积（提供半径即可）。
   - 其他数学计算或单位转换。

3. **学习与知识**：
   - 解答问题或提供学习资料。
   - 帮助理解复杂的概念。

4. **日常助手**：
   - 提醒事项（需要具体时间）。
   - 提供建议（比如旅行、饮食、健康等）。

5. **娱乐**：
   - 讲笑话或有趣的故事。
   - 推荐电影、书籍或音乐。

如果你有任何具体需求，随时告诉我，我会尽力帮你完成！

👤 用户: 现在几点了？顺便告诉我深圳的天气
🤖 助手: 你好！我可以帮助你完成很多事情，以下是一些我可以提供的服务：

1. **信息查询**：
   - 获取当前的日期和时间。
   - 查询天气信息（告诉我城市名称即可）。
   - 搜索互联网上的相关信息（比如新闻、百科等）。

2. **计算工具**：
   - 计算圆的面积（提供半径即可）。
   - 其他数学计算或单位转换。

3. **学习与知识**：
   - 解答问题或提供学习资料。
   - 帮助理解复杂的概念。

4. **日常助手**：
   - 提醒事项（需要具体时间）。
   - 提供建议（比如旅行、饮食、健康等）。

5. **娱乐**：
   - 讲笑话或有趣的故事。
   - 推荐电影、书籍或音乐。

如果你有任何具体需求，随时告诉我，我会尽力帮你完成！

👤 用户: 现在几点了？顺便告诉我深圳的天气
🔧 检测到 2 个工具调用
📞 调用工具: get_current_time
📝 参数: {}
✅ 工具执行结果: {'success': True, 'result': '2025-05-27 14:17:16'}
📞 调用工具: get_weather_info
📝 参数: {'cit

## 7. 高级功能：批量处理

演示批量处理多个请求的功能。

In [ ]:
async def batch_processing_demo():
    """批量处理演示"""
    
    caller = DeepSeekFunctionCaller(
        api_key=DEEPSEEK_API_KEY,
        base_url=DEEPSEEK_BASE_URL,
        model=DEEPSEEK_MODEL
    )
    
    # 批量任务
    batch_tasks = [
        "告诉我现在的时间",
        "计算半径为3的圆的面积", 
        "上海的天气如何？",
        "搜索Python编程相关信息",
        "计算半径为7的圆的面积，然后告诉我北京天气"
    ]
    
    print("🔄 开始批量处理演示")
    print(f"📝 总共 {len(batch_tasks)} 个任务")
    print("=" * 60)
    
    results = []
    
    try:
        for i, task in enumerate(batch_tasks, 1):
            print(f"\n📋 任务 {i}/{len(batch_tasks)}: {task}")
            
            messages = [{"role": "user", "content": task}]
            result = await caller.chat_with_functions(messages)
            
            if "error" in result:
                print(f"❌ 失败: {result['error']}")
                results.append({"task": task, "success": False, "error": result['error']})
            else:
                print(f"✅ 完成: {result['response']}")
                results.append({
                    "task": task, 
                    "success": True, 
                    "response": result['response'],
                    "function_calls": result['function_calls_used']
                })
        
        # 总结
        print(f"\n📊 批量处理总结:")
        print(f"✅ 成功: {sum(1 for r in results if r['success'])} 个")
        print(f"❌ 失败: {sum(1 for r in results if not r['success'])} 个")
        print(f"🔧 总函数调用: {sum(r.get('function_calls', 0) for r in results)} 次")
        
    finally:
        await caller.close()

# 运行批量处理演示
await batch_processing_demo()

⚠️ 警告: 未完全配置备用 API 客户端。如果主 API 失败，将无法进行回退。
🔄 开始批量处理演示
📝 总共 5 个任务

📋 任务 1/5: 告诉我现在的时间
🔧 检测到 1 个工具调用
📞 调用工具: get_current_time
📝 参数: {}
✅ 工具执行结果: {'success': True, 'result': '2025-05-27 14:17:38'}
🔧 检测到 1 个工具调用
📞 调用工具: get_current_time
📝 参数: {}
✅ 工具执行结果: {'success': True, 'result': '2025-05-27 14:17:38'}
✅ 完成: 现在是2025年5月27日，下午2点17分。

📋 任务 2/5: 计算半径为3的圆的面积
✅ 完成: 现在是2025年5月27日，下午2点17分。

📋 任务 2/5: 计算半径为3的圆的面积
🔧 检测到 1 个工具调用
📞 调用工具: calculate_circle_area
📝 参数: {'radius': 3}
✅ 工具执行结果: {'success': True, 'result': {'area': 28.274333882308138}}
🔧 检测到 1 个工具调用
📞 调用工具: calculate_circle_area
📝 参数: {'radius': 3}
✅ 工具执行结果: {'success': True, 'result': {'area': 28.274333882308138}}
✅ 完成: 半径为3的圆的面积约为28.27平方单位。

📋 任务 3/5: 上海的天气如何？
✅ 完成: 半径为3的圆的面积约为28.27平方单位。

📋 任务 3/5: 上海的天气如何？
🔧 检测到 1 个工具调用
📞 调用工具: get_weather_info
📝 参数: {'city': '上海'}
✅ 工具执行结果: {'success': True, 'result': '多云，温度 18°C'}
🔧 检测到 1 个工具调用
📞 调用工具: get_weather_info
📝 参数: {'city': '上海'}
✅ 工具执行结果: {'success': True, 'result': '多云，温度 18°C'}
✅ 完成: 上海的

## 8. 总结

这个 notebook 演示了如何使用 DeepSeek V3 模型实现传统的 function call 功能，包括：

1. ✅ **基本 API 连接测试** - 验证 API 配置正确性
2. 🛠️ **函数定义** - 创建可供模型调用的函数
3. 📋 **函数规范** - 按传统 Function Call 格式定义函数规范
4. 🤖 **核心实现** - DeepSeekFunctionCaller 类
5. 🧪 **功能测试** - 测试各种 function call 场景
6. 💬 **交互式对话** - 演示连续对话中的函数调用
7. 📦 **批量处理** - 演示批量任务处理能力

### 主要特性:
- 🔄 自动检测和执行函数调用
- 🛡️ 错误处理和重试机制
- 📝 完整的对话历史管理
- 🎯 支持多轮对话中的函数调用
- 📊 详细的执行统计信息

### Function Call 特点:

**传统 Function Call 方式**:
- 🔧 使用 `functions` 和 `function_call` 参数
- 📞 响应中使用 `message.function_call`
- 🔄 每次只能调用一个函数
- 📋 使用 `role: "function"` 返回函数结果

### Function Call 格式:

1. **API 调用参数**:
   ```python
   functions=FUNCTIONS,      # 函数定义列表
   function_call="auto"      # 自动决定是否调用函数
   ```

2. **函数定义格式**:
   ```python
   {
       "name": "function_name",
       "description": "函数描述",
       "parameters": {
           "type": "object",
           "properties": {...},
           "required": [...]
       }
   }
   ```

3. **响应处理**:
   ```python
   message.function_call.name        # 函数名
   message.function_call.arguments   # 函数参数 (JSON 字符串)
   ```

### 可扩展的功能:
- 添加更多自定义函数
- 实现函数调用缓存
- 添加函数调用权限控制
- 集成外部 API 服务
- 优化函数调用性能